In [1]:
import polars as pl
import os
import duckdb as db

# Chemin data

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data = os.path.join(parent_dir, "data")
data_path_equipement = os.path.join(data, "BPE23.parquet")

# BDD

In [3]:
df = pl.scan_parquet(data_path_equipement)

In [29]:
df_filtered = (
    df.filter(
        (pl.col("DOM") == "D") & 
        (pl.col("TYPEQU").str.contains("D")) &
        (pl.col("LATITUDE").is_not_null() & pl.col("LONGITUDE").is_not_null()) &
        (pl.col("LATITUDE") > 41) & (pl.col("LATITUDE") < 52) &
        (pl.col("LONGITUDE") > -5) & (pl.col("LONGITUDE") < 9) &
        (pl.col("SDOM").is_in(["D1", "D2", "D3"]))
    )
    .with_columns(pl.col("LIBCOM").str.to_lowercase())
    .select([
        "DOM", "SDOM", "TYPEQU", "DENS3", "DENS7",
        "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
        "CODPOS", "DEP", "REG", "LIBCOM"
    ])
    .collect()
)


In [5]:
display(df_filtered)

DOM,SDOM,TYPEQU,DENS3,DENS7,AAV2020,QP,QP2015,LATITUDE,LONGITUDE,CODPOS,DEP,REG,LIBCOM
str,str,str,str,str,str,str,str,f64,f64,str,str,str,str
"""D""","""D1""","""D101""","""1""","""1""","""001""","""CSZ""","""CSZ""",48.859152,2.341102,"""75001""","""75""","""11""","""paris 1er arrondissement"""
"""D""","""D1""","""D108""","""1""","""1""","""001""","""CSZ""","""CSZ""",48.865472,2.334304,"""75001""","""75""","""11""","""paris 1er arrondissement"""
"""D""","""D1""","""D108""","""1""","""1""","""001""","""CSZ""","""CSZ""",48.859328,2.346075,"""75001""","""75""","""11""","""paris 1er arrondissement"""
"""D""","""D1""","""D108""","""1""","""1""","""001""","""CSZ""","""CSZ""",48.863354,2.349947,"""75001""","""75""","""11""","""paris 1er arrondissement"""
"""D""","""D1""","""D108""","""1""","""1""","""001""","""CSZ""","""CSZ""",48.863715,2.347963,"""75001""","""75""","""11""","""paris 1er arrondissement"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""D""","""D2""","""D244""","""3""","""5""","""495""","""CSZ""","""CSZ""",42.621339,8.945191,"""20220""","""2B""","""94""","""monticello"""
"""D""","""D2""","""D244""","""3""","""5""","""495""","""CSZ""","""CSZ""",42.631096,8.950363,"""20220""","""2B""","""94""","""monticello"""
"""D""","""D2""","""D244""","""3""","""6""","""430""","""CSZ""","""CSZ""",42.546474,8.913547,"""20225""","""2B""","""94""","""muro"""


In [6]:
df_filtered.select(["LATITUDE", "LONGITUDE"])

LATITUDE,LONGITUDE
f64,f64
48.859152,2.341102
48.865472,2.334304
48.859328,2.346075
48.863354,2.349947
48.863715,2.347963
…,…
42.621339,8.945191
42.631096,8.950363
42.546474,8.913547


Les équipements de santé sont dans les variables :
- DOM : D
- SDOM : D1, D2, D3
- TYPEQU : D101, D102, D103, D108, D113

Variables importantes :
- DOM : Domaine d’appartenance de l’équipement
- SDOM : Sous-domaine d’appartenance de l’équipement
- TYPEQU : Type d’équipement 
- DENS3 : Grille communale de densité à 3 niveaux
- DENS7 : Grille communale de densité à 7 niveaux
- AAV2020 : Zonage en aire d’attraction des villes 2020 d’implantation de l’équipement
- QP : Quartier prioritaire de la politique de la ville d’appartenance de l’équipement
- QP2015 : Quartier prioritaire de la politique de la ville 2015 d’appartenance de l’équipement
- LATITUDE : Latitude en coordonnées GPS de l’équipement (degrés décimaux)
- LONGITUDE : Longitude en coordonnées GPS de l’équipement (degrés décimaux)
- CODPOS : Code postal de l’adresse d’implantation de l’équipement
- LIBCOM : Libellé de la commune d’implantation de l’équipement

# Fusion avec BDD des villes bénéficiaires

In [7]:
data_path_vb = os.path.join(data, "programme-petites-villes-de-demain-liste-des-villes-beneficiaires.csv")

In [8]:
df_vb = pl.scan_csv(data_path_vb, separator=";",
                    schema_overrides={
                        "Code Officiel Département": pl.Utf8,
                    })

In [9]:
df_vb.collect_schema().names()

['insee_com',
 'lib_com',
 'id_pvd',
 'date_signature',
 'Nom Officiel Région',
 'Code Officiel Département',
 'Nom Officiel Département',
 'Nom Officiel EPCI',
 'Code Officiel Région',
 'Code Officiel EPCI',
 'Geo Point',
 'Geo Shape']

In [45]:
df_vb_filtered = (
    df_vb.rename({
        "lib_com" : "LIBCOM",
        "Code Officiel Département" : "DEP",
        "Code Officiel Région" : "REG"
    })
    .with_columns([
        pl.col("Geo Point").str.split(", ").list.get(0).cast(pl.Float64).alias("LATITUDE"),
        pl.col("Geo Point").str.split(", ").list.get(1).cast(pl.Float64).alias("LONGITUDE"),
        pl.col("REG").cast(str),
        pl.col("LIBCOM").str.to_lowercase()
    ])
    .filter(
        (pl.col("LATITUDE") > 41) & (pl.col("LATITUDE") < 52) &
        (pl.col("LONGITUDE") > -5) & (pl.col("LONGITUDE") < 9),
    )
    
    .select(
        ["LIBCOM", "DEP", "REG", "LATITUDE", "LONGITUDE"]
    )
    .collect()
)


In [37]:
type(df_vb_filtered)

polars.dataframe.frame.DataFrame

In [ ]:
df_vb_sante = df_filtered.join(
    other = df_vb_filtered, 
    on =["LIBCOM", "DEP", "REG"],  
    how = "inner"
)

In [52]:
df_vb_sante

DOM,SDOM,TYPEQU,DENS3,DENS7,AAV2020,QP,QP2015,LATITUDE,LONGITUDE,CODPOS,DEP,REG,LIBCOM,LATITUDE_right,LONGITUDE_right
str,str,str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64
"""D""","""D2""","""D201""","""3""","""5""","""001""","""CSZ""","""CSZ""",48.413538,3.237936,"""77480""","""77""","""11""","""bray-sur-seine""",48.414018,3.242865
"""D""","""D2""","""D201""","""3""","""5""","""001""","""CSZ""","""CSZ""",48.41557,3.239829,"""77480""","""77""","""11""","""bray-sur-seine""",48.414018,3.242865
"""D""","""D2""","""D221""","""3""","""5""","""001""","""CSZ""","""CSZ""",48.41557,3.239829,"""77480""","""77""","""11""","""bray-sur-seine""",48.414018,3.242865
"""D""","""D2""","""D231""","""3""","""5""","""001""","""CSZ""","""CSZ""",48.41557,3.239829,"""77480""","""77""","""11""","""bray-sur-seine""",48.414018,3.242865
"""D""","""D2""","""D233""","""3""","""5""","""001""","""CSZ""","""CSZ""",48.415632,3.2373,"""77480""","""77""","""11""","""bray-sur-seine""",48.414018,3.242865
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""D""","""D3""","""D302""","""3""","""5""","""495""","""CSZ""","""CSZ""",42.627619,8.937875,"""20220""","""2B""","""94""","""l'île-rousse""",42.631678,8.933932
"""D""","""D3""","""D303""","""3""","""5""","""495""","""CSZ""","""CSZ""",42.628741,8.924511,"""20220""","""2B""","""94""","""l'île-rousse""",42.631678,8.933932
"""D""","""D3""","""D307""","""3""","""5""","""495""","""CSZ""","""CSZ""",42.633854,8.938551,"""20220""","""2B""","""94""","""l'île-rousse""",42.631678,8.933932


In [51]:
df_vb_sante.unique("LIBCOM").shape

(1596, 16)

# Groupby pour savoir cb d'équipement sont dans chaque ville 

In [63]:
df_vb_sante.group_by(
    by = "LIBCOM"
).agg(
    pl.col("DOM").count().alias("count_DOM")
).sort("count_DOM", descending=True)

by,count_DOM
str,u32
"""auray""",258
"""challans""",233
"""sallanches""",209
"""l'isle-sur-la-sorgue""",199
"""albertville""",199
…,…
"""banassac-canilhac""",2
"""cayres""",2
"""bel-air-val-d'ance""",1


# DuckDB

In [ ]:
df = pl.scan_parquet(data_path_equipement)

In [5]:
df_filtered = df.select([
    "DOM", "SDOM", "TYPEQU", "CAPACITE", "ACCES_SANITAIRE", "PRES_SANITAIRE",
    "DENS3", "DENS7", "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
    "CODPOS", "LIBCOM"
])


In [6]:
df_filtered = df_filtered.collect()

In [7]:
con = db.connect(database=':memory:')
con.register('df', df_filtered)

In [8]:
query = """
SELECT *
FROM df
WHERE
    DOM = 'D'
    OR SDOM IN ('D1', 'D2', 'D3')
    OR TYPEQU IN ('D101', 'D102', 'D103', 'D108', 'D113')
    OR ACCES_SANITAIRE = '1'
    OR PRES_SANITAIRE = '1'
"""

In [9]:
df_sante = con.execute(query).fetchdf()

: 